### Test Producer

`*`First need to start consumer in ./src/stream_process/kafka_message_sender.py
(Creating an ipython notebook for that too)

In [1]:
from kafka import KafkaProducer
import sys
from datetime import datetime
import time
import pandas as pd
import yaml
import sys
import signal

import random
import os.path

In [2]:
def yaml_loader(yaml_file):
	# os.path.dirname(os.path.realpath(__file__))
	with open(yaml_file) as yml:
		config = yaml.load(yml)
	return config

In [3]:
def load_csv(path_to_data,data_columns):
	nchunk_data = 1
	# read csv as a pandas DataFrame, filtering invalid data and using only the specified columns
	data_file = pd.read_csv(path_to_data,error_bad_lines=False,sep=',',header=0, iterator=True,chunksize = nchunk_data)
	return data_file

In [4]:
def signal_handler(signal,frame):
	print "--"
	print "--- kafka producer has been halted ---"
	print "--"
	sys.exit(0)

In [5]:
# path to config file
config_source = "config/producer_config.yml"

# load configuration parameters
config = yaml_loader(config_source)

# initialize parameters
topics = config['topics']
time_buffer = config['time_buffer']
data_source = config['data_file_name']
data_columns = config['data_column_keys']
port = config['port']
print port

ec2-34-193-153-112.compute-1.amazonaws.com:9092


MASTER NODE:
    Hostname:   ip-172-31-1-41
    Public DNS: ec2-34-198-103-9.compute-1.amazonaws.com

WORKER NODE:
    Hostname:   ip-172-31-1-40
    Public DNS: ec2-34-198-102-254.compute-1.amazonaws.com

WORKER NODE:
    Hostname:   ip-172-31-1-39
    Public DNS: ec2-34-198-22-65.compute-1.amazonaws.com

WORKER NODE:
    Hostname:   ip-172-31-1-46
    Public DNS: ec2-34-198-102-159.compute-1.amazonaws.com

In [6]:
# path to data file
path_to_data = os.path.abspath(os.path.join(os.getcwd(), '..',data_source))
# load data from directory 
data_file = load_csv(path_to_data,data_columns)
# create a producer object for the given ip
producer = KafkaProducer(bootstrap_servers=[port])

In [7]:
print producer

In [8]:
# run unless interrupted
while True:
    # iterate through DataFrame every second
    for chunk in data_file:

        # check for stopping input
        signal.signal(signal.SIGINT , signal_handler)

        # append current time to Pandas DataFrame
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        # current_time = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        time_dataframe  = pd.DataFrame({'timestamp':[current_time]})

        # select a random topic
        rand = random.randint(0,len(topics)-1) 
        print topics[rand]

        # convert DataFrame to json string 
        message = (chunk.join(time_dataframe)).to_json(double_precision=15,orient='records')

        print message.strip('[]').splitlines()[0]
        # send messages
        producer.send(topics[rand], value = message.strip('[]').splitlines()[0])
        #print topics[rand]

        # block until all async messages are sent
        producer.flush()

        # buffer time 
        time.sleep(time_buffer)

users
{"vendor_id":null,"pickup_datetime":null,"dropoff_datetime":null,"passenger_count":null,"trip_distance":null,"pickup_longitude":null,"pickup_latitude":null,"rate_code":null,"store_and_fwd_flag":null,"dropoff_longitude":null,"dropoff_latitude":null,"payment_type":null,"fare_amount":null,"surcharge":null,"mta_tax":null,"tip_amount":null,"tolls_amount":null,"total_amount":null,"timestamp":"2017-01-31 22:15:41.621"}
users
{"vendor_id":"VTS","pickup_datetime":"2013-05-01 00:04:00","dropoff_datetime":"2013-05-01 00:12:00","passenger_count":1,"trip_distance":1.34,"pickup_longitude":-73.982287,"pickup_latitude":40.772815000000001,"rate_code":1,"store_and_fwd_flag":null,"dropoff_longitude":-73.98621,"dropoff_latitude":40.758741999999998,"payment_type":"CSH","fare_amount":7,"surcharge":0.5,"mta_tax":0.5,"tip_amount":0,"tolls_amount":0,"total_amount":8,"timestamp":"2017-01-31 22:15:42.634"}
users
{"vendor_id":"VTS","pickup_datetime":"2013-05-01 00:03:00","dropoff_datetime":"2013-05-01 00:10

MemoryError: 

In [ ]:
print 1+1